# Estimación de máximo estiimador de verosimilitud del parámetro $\lambda$ de una distribución logarítmica

Para la solución del problema se hace uso del método de Newton-Raphson, cuyo implementación se muestra a continuación.

## Líbrerias y clases que se usaran

In [3]:
import math
import sympy
import numpy as np
import plotly.graph_objects as go

class RealFunction:

    def __call__(self, x):
        return self._evaluate(x, self._param)

    def __init__(self,  f, param = []):
        self._param = param
        self._evaluate = f
        self._der = dict()

## Algoritmo Newton-Raphson

La función siguiente función emplea el método Newton-Rapshon para calcular las raices de una función $f$. Como parametros necesito la función $f$, el punto inicial $x0$, la cantidad de iteraciones máximas $iterMax$, la tolerancia $tol$, y opcionalmente el minimo y máximo valor donde buscar como $tmin$ y $tmax$ respectivamente.

In [4]:
def NewtonRaphson(x0, f, derf, iterMax, tol, param = [], tmin = None, tmax = None):
    xk  = x0
    res = 0
    for k in range(iterMax):
        fk = f(xk, param)
        #print(k,xk,fk)
        if fk<tol and fk>-tol:
            res = 1
            break
        else:
            dfk = derf(xk, param)
            if dfk!=0:
                xk = xk - fk/dfk
                if(tmin != None):
                    xk = max(xk, tmin)
                if(tmax != None):
                    xk = min(xk, tmax)
            else:
                res = -1
                break
    return xk,k,res

Impresión de resultados del algoritmo Newton-Raphson

In [5]:
def interpretador_newton(f, derf, x0, iterMax, tol, param = []):
    xk,k,res = NewtonRaphson(x0 = x0, iterMax = iterMax, f = f, derf = derf, tol = tol, param = param)
    
    print("Para la funcion real f(x)\nCon punto inicial x0 = {}, donde f(x0) = {}".format(x0, f(x0,param)))

    print("El metodo de Newton, obtuvo res = {}, es decir:".format(res))
    if res==1:
        print('\tEncontro una raiz cerca de xk = {}\n\tdonde f(xk) = {},\n\ten {} iteraciones'.format(xk,f(xk,param), k))
    elif res==0:
        print('\tNo converje.')
        return
    else:
        print('\tSe cancelo la derivada en algun momento')
        return
    print("")

    return xk, res

## Intervalo de verosimilitud

Recibe como parámetros una función de verosimilitud $L$, el porcentaje $p$, el $emv$ de $L$, la cantidad de pasos para calcularlo $iterMax$ y la tolerancia de error $tol$. Imprime el intervalo de verosimilitud del $p\%$.

In [6]:
def likelihood_interval_mountain(l, sc, p, emv, iterMax, tol, param = []):
    
    g = lambda x, param : l(x,param) - l(emv,param) - math.log(p)
    derg = lambda x, param : sc(x,param)

    # x = np.linspace(10,100,iterMax)
    # fig = go.Figure()
    # fig.add_trace(
    #     go.Scatter(
    #         x=x,y=[g(x,param) for x in x],
    #         mode="lines", name=r'$\lambda$'
    #     )
    # )
    # fig.show()

    # x = np.linspace(10,100,iterMax)
    # fig = go.Figure()
    # fig.add_trace(
    #     go.Scatter(
    #         x=x,y=[derg(x,param) for x in x],
    #         mode="lines", name=r'$\lambda$'
    #     )
    # )
    # fig.show()
    xk,k,res = NewtonRaphson(emv + 1, g, derg, iterMax, tol, param, tmin = emv)
    R_lambda_tmax = xk
    xk,k,res = NewtonRaphson(emv - 1, g, derg, iterMax, tol, param, tmin = 0.000001, tmax = emv)
    R_lambda_tmin = xk
    print("El intervalo de verosimilitud del {}% es: [{},{}]".format(p*100, R_lambda_tmin, R_lambda_tmax))

In [16]:
def p(x, a):
    return np.exp(-x/a)

def L(x,param):
    p_arr = np.vectorize(p)(np.full(param[3][0]+1,x),param[2])
    return sum(param[1]*np.log(p_arr)+(param[0]-param[1])*np.log(1-p_arr))

def Sc(x, param):
    p_arr = np.vectorize(p)(np.full(param[3][0]+1,x),param[2])
    return sum((param[0]*p_arr-param[1])/(param[2]*(1-p_arr)))

def I(x,param):
    p_arr = np.vectorize(p)(np.full(param[3][0]+1,x),param[2])
    return sum(p_arr*(param[0]-param[1])/(param[2]**2*(1-p_arr)**2))


## Código Prinicipal

Declaración de parametros y funciones para estimar $\hat{\lambda}$, frecuencias esperadas e intervalo de verosimilitud $p$.

In [18]:
from pprint import pprint

def c_1(frec):
    return sum((frec[0][j]+1)*frec[1][j] for j in range(len(frec[0])))
def c_2(frec):
    return sum(frec[1][j] for j in range(len(frec[0])))
    
def main():
    
    n = 5
    k = 9
    a = 2
    x = np.array([0, 0, 0, 0, 1, 2, 3, 3, 5, 5])
    param = np.array([np.full(k+1, n),x,np.array([a**i for i in range(k+1)]),np.full(k+1,k)])
    f = Sc
    derf = lambda x, param : -I(x,param)
    iterMax = int(1e3)
    tol = 1e-6
    x0 = 0.1

    emv_lambda, _ = interpretador_newton(f, derf, x0, iterMax, tol, param = param)
    
    print(f"La matriz de información observada es: {derf(emv_lambda, param)}")

    likelihood_interval_mountain(L, Sc, 0.25, emv_lambda, iterMax, tol, param)

    var_emv = -1/derf(emv_lambda, param)

    print(f"Varianza del emv es: {var_emv}\n")
    
    z_a2 = 1.96
    print(f"Intervalo del 95% de confianza es: [{emv_lambda-z_a2*(var_emv)**(1/2)},{emv_lambda+z_a2*(var_emv)**(1/2)}]")



In [19]:
main()

Para la funcion real f(x)
Con punto inicial x0 = 0.1, donde f(x0) = 304.94806896962115
El metodo de Newton, obtuvo res = 1, es decir:
	Encontro una raiz cerca de xk = 30.649790412939417
	donde f(xk) = 6.453726442146035e-12,
	en 14 iteraciones

La matriz de información observada es: -0.012249045324417698
El intervalo de verosimilitud del 25.0% es: [18.44784413990403,49.12870854510824]
Varianza del emv es: 81.6390154101694

Intervalo del 95% de confianza es: [12.940345430114384,48.35923539576445]
